<a href="https://colab.research.google.com/github/suin8606/Soccer_data/blob/main/Database_Population_%26_Querying.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Database Population & Querying

Using Pandas & SQLAlchemy to store and retrieve StatsBomb event data


In [5]:
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
from sqlalchemy import create_engine

In [3]:
base_url = "https://raw.githubusercontent.com/statsbomb/open-data/master/data/"
comp_url = base_url + "matches/{}/{}.json"
match_url = base_url + "events/{}.json"

In [7]:
def parse_data(competition_id, season_id):
    matches=requests.get(url=comp_url.format(competition_id,season_id)).json()
    match_ids= [x["match_id"]for x in matches]
    events=[]
    for match_id in tqdm(match_ids):
        for y in requests.get(url=match_url.format(match_id)).json():
            events.append(y)
    return pd.json_normalize(events, sep='_')

In [8]:
competition_id = 43
season_id = 3
df = parse_data(competition_id, season_id)

100%|██████████| 64/64 [00:31<00:00,  2.04it/s]


In [9]:
df.head()

,id,index,period,timestamp,minute,second,possession,duration,type_id,type_name,...,pass_miscommunication,pass_technique_id,pass_technique_name,pass_through_ball,shot_redirect,miscontrol_aerial_won,block_save_block,foul_committed_penalty,foul_won_penalty,shot_follows_dribble
0,14ca13b2-3490-4916-831d-d0b54aec1a8a,1,1,00:00:00.000,0,0,1,0.000,35,Starting XI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a95529be-109f-476c-8bdb-a18376072dbf,2,1,00:00:00.000,0,0,1,38.786,35,Starting XI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,aa238766-3417-4871-8fe0-11485a47a37d,3,1,00:00:00.000,0,0,1,NaN,18,Half Start,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15480c9d-f4c8-4fef-911e-e5504c212e36,4,1,00:00:00.000,0,0,1,NaN,18,Half Start,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,c6758a85-e41e-430c-85f0-8646bfc26f31,5,1,00:00:01.080,0,1,2,2.760,30,Pass,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df.columns.values

array(['id', 'index', 'period', 'timestamp', 'minute', 'second',
       'possession', 'duration', 'type_id', 'type_name',
       'possession_team_id', 'possession_team_name', 'play_pattern_id',
       'play_pattern_name', 'team_id', 'team_name', 'tactics_formation',
       'tactics_lineup', 'related_events', 'location', 'player_id',
       'player_name', 'position_id', 'position_name', 'pass_recipient_id',
       'pass_recipient_name', 'pass_length', 'pass_angle',
       'pass_height_id', 'pass_height_name', 'pass_end_location',
       'pass_type_id', 'pass_type_name', 'pass_body_part_id',
       'pass_body_part_name', 'under_pressure', 'carry_end_location',
       'pass_outcome_id', 'pass_outcome_name', 'ball_receipt_outcome_id',
       'ball_receipt_outcome_name', 'duel_type_id', 'duel_type_name',
       'pass_aerial_won', 'counterpress',
       'ball_recovery_recovery_failure', 'pass_switch',
       'dribble_outcome_id', 'dribble_outcome_name', 'duel_outcome_id',
       'duel_outcom

In [16]:
location_columns=[x for x in df.columns.values if 'location' in x]
location_columns

['location',
 'pass_end_location',
 'carry_end_location',
 'shot_end_location',
 'goalkeeper_end_location']

In [26]:
for col in location_columns:
    print(col)
    for i,dimention in enumerate(["x","y"]):
      print(i)

location
0
1
pass_end_location
0
1
carry_end_location
0
1
shot_end_location
0
1
goalkeeper_end_location
0
1


In [33]:
for col in location_columns:
    for i,dimention in enumerate(["x","y"]):
        new_col=col.replace("location",dimention)
        print(new_col)
        df[new_col]=df.apply(lambda x: x[col][i] if type(x[col])==list else None, axis=1)

x
y
pass_end_x
pass_end_y
carry_end_x
carry_end_y
shot_end_x
shot_end_y
goalkeeper_end_x
goalkeeper_end_y


In [34]:
df.columns.values

array(['id', 'index', 'period', 'timestamp', 'minute', 'second',
       'possession', 'duration', 'type_id', 'type_name',
       'possession_team_id', 'possession_team_name', 'play_pattern_id',
       'play_pattern_name', 'team_id', 'team_name', 'tactics_formation',
       'tactics_lineup', 'related_events', 'location', 'player_id',
       'player_name', 'position_id', 'position_name', 'pass_recipient_id',
       'pass_recipient_name', 'pass_length', 'pass_angle',
       'pass_height_id', 'pass_height_name', 'pass_end_location',
       'pass_type_id', 'pass_type_name', 'pass_body_part_id',
       'pass_body_part_name', 'under_pressure', 'carry_end_location',
       'pass_outcome_id', 'pass_outcome_name', 'ball_receipt_outcome_id',
       'ball_receipt_outcome_name', 'duel_type_id', 'duel_type_name',
       'pass_aerial_won', 'counterpress',
       'ball_recovery_recovery_failure', 'pass_switch',
       'dribble_outcome_id', 'dribble_outcome_name', 'duel_outcome_id',
       'duel_outcom

In [35]:
df=df[[x for x in df.columns if x not in location_columns]]

In [38]:
columns_remove=['tactics_lineup', 'related_events', 'shot_freeze_frame']
df=df[[x for x in df.columns if x not in columns_remove]]

In [40]:
engine = create_engine('sqlite://')

In [41]:
df.to_sql('events', engine)

In [44]:
top_passers = ("""
Select player_name, count(*) as passes
From events
Where 1=1 and type_name = "Pass"
Group by player_id
Order by count(*) desc
""")
pd.read_sql(top_passers, engine).head(10)

,player_name,passes
0,Luka Modrić,527
1,Sergio Ramos García,496
2,John Stones,479
3,Francisco Román Alarcón Suárez,475
4,Toby Alderweireld,446
5,Harry Maguire,434
6,Ivan Rakitić,426
7,Jordi Alba Ramos,414
8,Kieran Trippier,393
9,Kyle Walker,385


In [49]:
top_xg = ("""
Select player_name, round(sum(shot_statsbomb_xg),2) as 'total xg'
From events
Where 1=1 and type_name = "Shot"
Group by player_id
Order by 2 desc
""")
pd.read_sql(top_xg, engine).head(10)

,player_name,total xg
0,Neymar da Silva Santos Junior,4.53
1,Harry Kane,4.33
2,Luka Modrić,3.36
3,Antoine Griezmann,3.35
4,Eden Hazard,3.14
5,Ivan Rakitić,2.89
6,Cristiano Ronaldo dos Santos Aveiro,2.63
7,Romelu Lukaku Menama,2.58
8,Michy Batshuayi Tunga,2.42
9,Diego da Silva Costa,2.38
